# List files from Cloud Storage

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
from google.cloud import storage
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
def list_blobs(bucket_name, prefix):
    """Lists all the blobs in the bucket."""
    images_paths = []

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)

    for blob in blobs:
        images_paths.append(blob.name)
        
    return images_paths

In [7]:
prefix = 'datasets/xray-chest-nih/images'
bucket_name = 'tensorflow-samples'

In [ ]:
images_paths = list_blobs(bucket_name, prefix)

# Filter Images Paths to create Dataset

In [8]:
images_prefix = 'gs://tensorflow-samples/datasets/xray-chest-nih/images/'

In [9]:
labels_path = 'gs://tensorflow-samples/datasets/xray-chest-nih/labels/labels.csv'
df_files = pd.read_csv(labels_path)

In [10]:
# Filter images with label Effusion
effusion_images = df_files[df_files['Finding Labels'].str.contains('Effusion')]

# Keep only image name and label
effusion_images = effusion_images[['Image Index','Finding Labels']]
effusion_images['Image Index'] = images_prefix + effusion_images['Image Index']
effusion_images['Finding Labels'] = 1

In [11]:
# Filter images with label No Finding
nofindings_images = df_files[df_files['Finding Labels'].str.contains('No Finding')]

# Keep only image name and label
nofindings_images = nofindings_images[['Image Index','Finding Labels']]
nofindings_images['Image Index'] = images_prefix + nofindings_images['Image Index']
nofindings_images['Finding Labels'] = 0

In [12]:
# Only a subset of the main dataset
nofindings_images = nofindings_images.iloc[:20000]

#### Let's concatenate the lists

In [13]:
effusion_range = len(effusion_images)
nofindings_range = len(nofindings_images)

Train Images and Labels

In [14]:
train_images = list(effusion_images['Image Index'][:int(effusion_range*0.90)])
train_images = train_images + list(nofindings_images['Image Index'][:int(nofindings_range*0.90)])

In [15]:
train_labels = list(effusion_images['Finding Labels'][:int(effusion_range*0.90)]) 
train_labels = train_labels + list(nofindings_images['Finding Labels'][:int(nofindings_range*0.90)])

Val Images and Labels

In [16]:
val_images = list(effusion_images['Image Index'][int(effusion_range*0.90):int(effusion_range*0.97)])
val_images = val_images + list(nofindings_images['Image Index'][int(nofindings_range*0.90):int(nofindings_range*0.97)])

In [17]:
val_labels = list(effusion_images['Finding Labels'][int(effusion_range*0.90):int(effusion_range*0.97)]) 
val_labels = val_labels + list(nofindings_images['Finding Labels'][int(nofindings_range*0.90):int(nofindings_range*0.97)])

Test Images and Labels

In [18]:
test_images = list(effusion_images['Image Index'][int(effusion_range*0.97):])
test_images = test_images + list(nofindings_images['Image Index'][int(nofindings_range*0.97):])

In [19]:
test_labels = list(effusion_images['Finding Labels'][int(effusion_range*0.97):]) 
test_labels = test_labels + list(nofindings_images['Finding Labels'][int(nofindings_range*0.97):])

In [20]:
print(len(train_images))
print(len(val_images))
print(len(test_images))

29985
2332
1000


# Generate TFRecord with images

In [21]:
# First let's create a tf.data.dataset from file paths
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=len(train_images))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [22]:
def process_image_tfrecord(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_png(image)
    image = tf.image.resize(image, [512,512], method='nearest')
    image = tf.expand_dims(image[:,:,0], -1)
    image = tf.image.encode_png(image)
    
    return image, label

In [23]:
def build_dataset_tfrecord(paths_dataset):
    dataset = paths_dataset.map(process_image_tfrecord, num_parallel_calls=AUTOTUNE)    
    return dataset

In [24]:
def tf_serialize_example(image, label):
    
    def _bytes_feature(value):
        """Returns a bytes_list from a string / byte."""
        if isinstance(value, type(tf.constant(0))):
            value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    def _float_feature(value):
        """Returns a float_list from a float / double."""
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

    def _int64_feature(value):
        """Returns an int64_list from a bool / enum / int / uint."""
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))    
    
    def serialize_example(image, label):
        
        feature = {
            'image': _bytes_feature(image),
            'label': _int64_feature(label)
        }

        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
        
        return example_proto.SerializeToString()
    
    tf_string = serialize_example(image, label)

    return tf_string

In [25]:
# Create TFRecord with `n_shards` shards (GZIP)
def create_tfrecord(ds, n_shards, name):

    for i in range(n_shards):
        batch = map(lambda x: tf_serialize_example(x[0],x[1]), ds.shard(n_shards, i)
                    .apply(build_dataset_tfrecord)
                    .make_one_shot_iterator())
        
        with tf.io.TFRecordWriter('{name}-output_file-part-{i}.tfrecord'.format(i=i, name=name), 'GZIP') as writer:
            print('Creating TFRecord ... output_file-part-{i}.tfrecord'.format(i=i))
            for a in batch:
                writer.write(a)

In [ ]:
create_tfrecord(train_dataset, 30, 'train')
create_tfrecord(val_dataset, 6, 'val')
create_tfrecord(test_dataset, 4, 'test')

In [ ]:
!gsutil -m cp train*.tfrecord gs://tensorflow-samples/datasets/xray-chest-nih/tfrecords/train/
!gsutil -m cp val*.tfrecord gs://tensorflow-samples/datasets/xray-chest-nih/tfrecords/val/
!gsutil -m cp test*.tfrecord gs://tensorflow-samples/datasets/xray-chest-nih/tfrecords/test/

# Consume TFRecords and create tf.data.Dataset

In [26]:
TRAIN_TFRECORDS = 'gs://tensorflow-samples/datasets/xray-chest-nih/tfrecords/train/*'
VAL_TFRECORDS = 'gs://tensorflow-samples/datasets/xray-chest-nih/tfrecords/val/*'
TEST_TFRECORDS = 'gs://tensorflow-samples/datasets/xray-chest-nih/tfrecords/test/*'
BATCH_SIZE = 32
STEPS_TRAIN = int(len(train_images)/BATCH_SIZE)
STEPS_VAL = int(len(val_images)/BATCH_SIZE)
STEPS_TEST = int(len(test_images)/BATCH_SIZE)

In [27]:
@tf.function
def parse_function(example_proto):
    # Parse the input `tf.Example` proto using the dictionary above.
    
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    
    return tf.io.parse_example(example_proto, feature_description)

In [28]:
@tf.function
def process_image(record):
    image = tf.map_fn(tf.io.decode_png, record['image'], dtype=tf.uint8)
    image = tf.map_fn(lambda image: 
                      tf.image.convert_image_dtype(image, dtype=tf.float32), image, dtype=tf.float32)
    
    label = record['label']
    
    return image, label

In [29]:
# Convert grayscale => RGB to use InceptionV3
# NOT USED
@tf.function
def grayscale_to_rgb(images, labels):
    images = tf.image.grayscale_to_rgb(images)
    return images, labels

In [30]:
@tf.function
def get_tfrecord(filename):
    return tf.data.TFRecordDataset(filename, compression_type='GZIP', 
                                   num_parallel_reads=AUTOTUNE)

In [31]:
def build_dataset(dataset, batch_size=BATCH_SIZE):
    
    dataset = dataset.interleave(get_tfrecord, num_parallel_calls=AUTOTUNE)

    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(process_image, num_parallel_calls=AUTOTUNE)

    dataset = dataset.repeat()
    # Pipeline next iteration
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [32]:
train_files= tf.io.gfile.glob(TRAIN_TFRECORDS)
val_files= tf.io.gfile.glob(VAL_TFRECORDS)
test_files= tf.io.gfile.glob(TEST_TFRECORDS)

train_filenames_dataset = tf.data.Dataset.from_tensor_slices(train_files)
val_filenames_dataset = tf.data.Dataset.from_tensor_slices(val_files)
test_filenames_dataset = tf.data.Dataset.from_tensor_slices(test_files)

In [33]:
train_dataset = build_dataset(train_filenames_dataset)
val_dataset = build_dataset(val_filenames_dataset)
test_dataset = build_dataset(test_filenames_dataset)

# Train Model

In [34]:
IMG_SHAPE = (512,512,1)

In [155]:
# # Functional API
# def create_model(img_shape=IMG_SHAPE):
    
#     # Define input and shapes
#     img_inputs = tf.keras.Input(shape=img_shape)

#     # 1th group
#     x = tf.keras.layers.Conv2D(64, (5,5), use_bias=False)(img_inputs)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Activation('relu')(x)
#     x = tf.keras.layers.MaxPool2D((3,3),(2,2))(x)

#     # 2nd group
#     x = tf.keras.layers.Conv2D(128, (5,5), use_bias=False)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Activation('relu')(x)
#     x = tf.keras.layers.MaxPool2D((3,3),(2,2))(x)

#     # 3rd group
#     x = tf.keras.layers.Conv2D(256, (5,5), use_bias=False)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Activation('relu')(x)
#     x = tf.keras.layers.MaxPool2D((3,3),(2,2))(x)

#     # 4th group
#     x = tf.keras.layers.Conv2D(512, (5,5), use_bias=False)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Activation('relu')(x)
#     x = tf.keras.layers.MaxPool2D((3,3),(2,2))(x)

#     # 5th group
#     x = tf.keras.layers.GlobalAvgPool2D()(x)

#     # Classification
#     outputs = tf.keras.layers.Dense(1)(x)

#     # Create model
#     model = tf.keras.Model(img_inputs, outputs)

#     return model

In [35]:
# Sequential API
def create_model(img_shape=IMG_SHAPE):
    
    model = tf.keras.Sequential([
        # 1th group
        tf.keras.layers.Conv2D(64, (5,5), use_bias=False, input_shape=img_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D((3,3),(2,2)),

        # 2nd group
        tf.keras.layers.Conv2D(128, (5,5), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D((3,3),(2,2)),

        # 3rd group
        tf.keras.layers.Conv2D(256, (5,5), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D((3,3),(2,2)),

        # 4th group
        tf.keras.layers.Conv2D(512, (5,5), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D((3,3),(2,2)),

        # 5th group
        tf.keras.layers.GlobalAvgPool2D(),

        # Classification
        tf.keras.layers.Dense(1)
    ])
    
    return model

### Define Callbacks

In [36]:
model_ckp = tf.keras.callbacks.ModelCheckpoint(filepath='mymodel_{epoch}',
                                               save_best_only=True, monitor='val_loss', verbose=1)

In [37]:
def scheduler(epoch):
    if epoch < 6:
        return (epoch*0.1*(32/256)/5)
    elif epoch >= 6 and epoch < 30:
        return (0.1*(32/256))
    elif epoch >= 30 and epoch < 60:
        return (0.1*(32/256))/10
    elif epoch >= 60 and epoch < 90:
        return (0.1*(32/256))/100
    else:
        return (0.1*(32/256))/1000

In [38]:
lr_sched = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [39]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        optimizer = self.model.optimizer
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1, optimizer.lr.numpy()))

In [40]:
def compile_model(model):
    model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

# Fit model (Distributed?)

In [45]:
def model_strategy(distributed=False):
    if distributed:
        strategy = tf.distribute.MirroredStrategy()
        
        with strategy.scope():
            model = create_model()
            model = compile_model(model)
            
    else:
        model = create_model()
        model = compile_model(model)
    
    return model

In [41]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_model()
    model = compile_model(model)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [47]:
model = model_strategy(distributed=True)

In [52]:
model.load_weights('./mymodel')

In [44]:
history = model.fit(train_dataset, epochs=5, 
                    validation_data=val_dataset,
                    steps_per_epoch=STEPS_TRAIN,
                    validation_steps=STEPS_VAL,
                    callbacks=[PrintLR(), lr_sched, model_ckp])

Train for 937 steps, validate for 72 steps
Epoch 1/5
  3/937 [..............................] - ETA: 7:57:57 - loss: 0.7434 - acc: 0.4531
Learning rate for epoch 1 is 0.0


KeyboardInterrupt: 

# Model Evaluation

In [ ]:
model.evaluate(test_dataset, steps=STEPS_TEST)

# AI Explanations

In [115]:
## Convert our Keras model to an estimator and then export to SavedModel
keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir='export')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'export', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff95ff3d610>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [118]:
def decode_img_bytes(img_bytes, height, width, color_depth):
    features = tf.squeeze(img_bytes, axis=1, name='input_squeeze')
    float_pixels = tf.map_fn(
        lambda img_string: tf.io.decode_image(
            img_string, 
            channels=color_depth,
            dtype=tf.float32
        ),
        features,
        dtype=tf.float32,
        name='input_convert'
    )

    tf.Tensor.set_shape(float_pixels, (None, height, width, color_depth))
    float_pixels = tf.identity(float_pixels, name='input_pixels')

    return float_pixels

def serving_input_receiver_fn():
    img_bytes = tf.placeholder(shape=(None,1), dtype=tf.string)
    img_float = decode_img_bytes(img_bytes, 512, 512, 1)
    return tf.estimator.export.ServingInputReceiver({'input_1': img_float}, {'input_1': img_bytes})

In [120]:
BUCKET_NAME = 'tensorflow-samples/datasets/xray-chest-nih'
export_path = keras_estimator.export_saved_model(
  'gs://' + BUCKET_NAME + '/explanations',
  serving_input_receiver_fn
).decode('utf-8')
print("Model exported to: ", export_path)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from export/keras/keras_model.ckpt
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: gs://tensorflow-samples/datasets/xray-chest-nih/explanations/temp-b'1588181779'/saved_model.pb
Model exported to:  gs://tensorflow-samples/datasets/xray-chest-nih/explanations/1588181779


In [121]:
!saved_model_cli show --dir $export_path --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 1)
        name: Placeholder:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: dense/BiasAdd:0
  Method name is: tensorflow/serving/predict


In [123]:
explanation_metadata = {
    "inputs": {
      "data": {
        "input_tensor_name": "input_pixels:0",
        "modality": "image",
        "input_baselines": [0,1]
      }
    },
    "outputs": {
      "probability": {
        "output_tensor_name": "dense/BiasAdd:0"
      }
    },
  "framework": "tensorflow"
}

In [124]:
import json
# Write the json to a local file
with open('explanation_metadata.json', 'w') as output_file:
    json.dump(explanation_metadata, output_file)

In [125]:
# Copy this file into the GCS location with our SavedModel assets
!gsutil cp explanation_metadata.json $export_path

Copying file://explanation_metadata.json [Content-Type=application/json]...
/ [1 files][  207.0 B/  207.0 B]                                                
Operation completed over 1 objects/207.0 B.                                      


In [126]:
MODEL = 'xray_explain'

In [ ]:
# Create the model if it doesn't exist yet (you only need to run this once)
!gcloud ai-platform models create $MODEL --enable-logging --regions=us-central1

In [129]:
# Each time you create a version the name should be unique
IG_VERSION = 'v_07'

In [128]:
# Create the version with gcloud
!gcloud beta ai-platform versions create $IG_VERSION \
--model $MODEL \
--origin $export_path \
--runtime-version 1.15 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--explanation-method integrated-gradients \
--num-integral-steps 25

Explanations reflect patterns in your model, but don't necessarily reveal fundamental relationships about your data population. See https://cloud.google.com/ml-engine/docs/ai-explanations/limitations for more information.
Creating version (this might take a few minutes)......done.                    


In [130]:
# Make sure the IG model deployed correctly. State should be `READY` in the following log
!gcloud ai-platform versions describe $IG_VERSION --model $MODEL

createTime: '2020-04-29T16:54:47Z'
deploymentUri: gs://tensorflow-samples/datasets/xray-chest-nih/explanations/1588174120
etag: DDvhP0oohws=
explanationConfig:
  integratedGradientsAttribution:
    numIntegralSteps: 25
framework: TENSORFLOW
isDefault: true
lastUseTime: '2020-04-29T17:20:55Z'
machineType: n1-standard-4
name: projects/cool-ml-demos/models/xray_explain/versions/v_07
pythonVersion: '3.7'
runtimeVersion: '1.15'
state: READY


# Get Predictions and explanations on deployed model

In [95]:
!mkdir examples

In [142]:
!gsutil cp gs://tensorflow-samples/datasets/xray-chest-nih/images/00000003_000.png ./examples

Copying gs://tensorflow-samples/datasets/xray-chest-nih/images/00000003_000.png...
/ [1 files][439.5 KiB/439.5 KiB]                                                
Operation completed over 1 objects/439.5 KiB.                                    


In [131]:
import PIL
import os

In [132]:
from matplotlib import pyplot as plt
from base64 import b64encode

In [143]:
# Resize the images to what our model is expecting (192,192)
test_filenames = []

images = ['00000001_000.png','00000002_000.png','00000003_000.png']

for i in images:
    img_path = '/home/jupyter/ml-samples/googlecloud/explanations/xray/examples/' + i
    with PIL.Image.open(img_path) as ex_img:
        resize_img = ex_img.resize([512,512])
        resize_img.save(img_path)
        test_filenames.append(img_path)

In [144]:
# Prepare our prediction JSON to send to our Cloud model
instances = []
!rm xray-data.txt

for i in test_filenames:
    with open(i, 'rb') as example_img:
        b64str = b64encode(example_img.read()).decode('utf-8')
        with open('xray-data.txt', 'a') as outfile:
            json.dump({'input_1': [{'b64': b64str}]}, outfile)
            outfile.write('\n')
        instances.append({'input_1': [{'b64': b64str}]})

rm: cannot remove 'xray-data.txt': No such file or directory


## Make an AI Explanation request with gcloud

In [145]:
# IG EXPLANATIONS
ig_explanations = !gcloud beta ai-platform explain --model $MODEL --version $IG_VERSION --json-instances='xray-data.txt'
ig_response = json.loads(ig_explanations.s)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [153]:
predict = !gcloud beta ai-platform explain --model $MODEL --version $IG_VERSION --json-instances='xray-data.txt'

In [154]:
predict

['WARNING: Using endpoint [https://ml.googleapis.com/]',
 '{',
 '  "error": "Explainability failed with exception: <_InactiveRpcError of RPC that terminated with:\\n\\tstatus = StatusCode.UNAVAILABLE\\n\\tdetails = \\"upstream connect error or disconnect/reset before headers. reset reason: connection termination\\"\\n\\tdebug_error_string = \\"{\\"created\\":\\"@1588182760.633611324\\",\\"description\\":\\"Error received from peer ipv4:10.59.250.192:8500\\",\\"file\\":\\"src/core/lib/surface/call.cc\\",\\"file_line\\":1056,\\"grpc_message\\":\\"upstream connect error or disconnect/reset before headers. reset reason: connection termination\\",\\"grpc_status\\":14}\\"\\n>"',
 '}']